In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        #how many layers are there and for each layer, we create an attribute
        #input layer (4 features)
        super().__init__()
        #hidden layer 1 (h1, arbitrary)
        self.fc1=nn.Linear(in_features, h1)
        #hidden layer 2 (h2, arbitraty)
        self.fc2=nn.Linear(h1,h2)
        #output layer (3 neurons)
        self.out=nn.Linear(h2, out_features)
    def forward(self, x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.out(x)
        return x

In [6]:
torch.manual_seed(32)
model=Model()

In [16]:
import pandas as pd

In [17]:
df=pd.read_csv('C:\\Users\\DELL\\anaconda3\\pkgs\\bokeh-3.2.1-py311h746a85d_0\\Lib\\site-packages\\bokeh\\sampledata\\_data\\iris.csv')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [20]:
print(df)

     sepal_length  sepal_width  petal_length  petal_width    species
0             5.1          3.5           1.4          0.2     setosa
1             4.9          3.0           1.4          0.2     setosa
2             4.7          3.2           1.3          0.2     setosa
3             4.6          3.1           1.5          0.2     setosa
4             5.0          3.6           1.4          0.2     setosa
..            ...          ...           ...          ...        ...
145           6.7          3.0           5.2          2.3  virginica
146           6.3          2.5           5.0          1.9  virginica
147           6.5          3.0           5.2          2.0  virginica
148           6.2          3.4           5.4          2.3  virginica
149           5.9          3.0           5.1          1.8  virginica

[150 rows x 5 columns]


In [22]:
df['target']=df['species'].astype('category').cat.codes

In [23]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species,target
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,0
2,4.7,3.2,1.3,0.2,setosa,0
3,4.6,3.1,1.5,0.2,setosa,0
4,5.0,3.6,1.4,0.2,setosa,0


In [36]:
df.shape

(150, 5)

In [37]:
df.tail()

,sepal_length,sepal_width,petal_length,petal_width,target
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0
149,5.9,3.0,5.1,1.8,2.0


In [38]:
X=df.drop('target', axis=1)
y=df['target']

In [39]:
type(y)

pandas.core.series.Series

In [40]:
X=X.values
y=y.values

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2, random_state=33)

In [44]:
X_train=torch.FloatTensor(X_train)
X_test=torch.FloatTensor(X_test)

In [45]:
y_train=torch.LongTensor(y_train)
y_test=torch.LongTensor(y_test)

In [46]:

criterion=nn.CrossEntropyLoss()

optimizer=torch.optim.Adam(model.parameters(), lr=0.01) #lower the learning rate, the longer it takes to train the data

In [49]:
model.parameters()

<generator object Module.parameters at 0x000001EB373D2180>

In [50]:
model.parameters

<bound method Module.parameters of Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)>

In [51]:
#how many epochs? 
#1 epoch == 1 run through the entire training data 
epochs=100
losses=[]
for i in range (epochs):
    #forward through the network
    y_pred=model.forward(X_train)
    #now calc loss 
    loss=criterion(y_pred, y_train)
    losses.append(loss)
    if i%10==0:
        print(f'epoch {i} - loss is {loss}')
        
    #back propogation 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch 0 - loss is 1.1507450342178345
epoch 10 - loss is 0.9371448755264282
epoch 20 - loss is 0.7796242237091064
epoch 30 - loss is 0.6078532338142395
epoch 40 - loss is 0.39894530177116394
epoch 50 - loss is 0.2524919807910919
epoch 60 - loss is 0.14927689731121063
epoch 70 - loss is 0.10029555857181549
epoch 80 - loss is 0.08100692182779312
epoch 90 - loss is 0.07216036319732666


In [52]:
#this is the testing part. we don't have to do back propogation here 
with torch.no_grad():
    y_eval=model.forward(X_test)
    loss=criterion(y_eval, y_test)

In [53]:
loss

tensor(0.0582)

In [58]:
correct=0

with torch.no_grad():
    for i, data in enumerate (X_test):
        y_val=model.forward(data)
        print(f'{i+1} {str(y_val)} {y_test[i]}')  #the higher the value, the more the probability 
        
        if y_val.argmax().item()==y_test[i]:
            correct+=1
print(f'We got {correct} correct')

1 tensor([-2.1235,  4.8067, -0.8803]) 1
2 tensor([-1.7920,  5.3100, -1.5693]) 1
3 tensor([  6.3723,   0.8741, -10.0971]) 0
4 tensor([-3.9129,  4.5951,  1.1509]) 1
5 tensor([-7.4882,  3.1953,  5.7839]) 2
6 tensor([-10.5202,   1.6381,   9.6291]) 2
7 tensor([  6.3364,   1.0237, -10.1951]) 0
8 tensor([  7.0690,   0.7370, -10.9620]) 0
9 tensor([-7.2218,  3.3422,  5.3528]) 2
10 tensor([-9.4170,  2.5675,  8.1028]) 2
11 tensor([-9.9029,  2.3388,  8.7142]) 2
12 tensor([ 6.2942,  0.6938, -9.8046]) 0
13 tensor([-9.3335,  2.1817,  8.1917]) 2
14 tensor([-3.7832,  4.5046,  1.0603]) 1
15 tensor([-7.8793,  3.0060,  6.2225]) 2
16 tensor([-1.8810,  5.1571, -1.3572]) 1
17 tensor([-5.7107,  3.5003,  3.6612]) 2
18 tensor([  7.2014,   0.7687, -11.1842]) 0
19 tensor([-3.2961,  4.7939,  0.3307]) 1
20 tensor([-7.7822,  3.7560,  5.7040]) 2
21 tensor([  6.6703,   0.8191, -10.4707]) 0
22 tensor([  7.4580,   0.9259, -11.7103]) 0
23 tensor([-9.7801,  2.1658,  8.6656]) 2
24 tensor([  6.5976,   0.7715, -10.3186]) 0
2

In [59]:
torch.save(model.state_dict(), 'my_iris_model.pt')


In [60]:
new_model=Model() #this model has no idea what the weights and biases should be 
new_model.load_state_dict(torch.load('my_iris_model.pt'))


<All keys matched successfully>

In [62]:
new_model.eval()

Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)

In [63]:
mystery_iris=torch.tensor([5.6, 3.7, 2.2, 0.5])


In [66]:
with torch.no_grad():
    print(new_model(mystery_iris)) #you can use new_model or the old one, they're essentially the same 
    print(new_model(mystery_iris).argmax())

tensor([  5.9874,   1.5769, -10.0753])
tensor(0)
